### Connecting to the DataBase

In [62]:
import mysql.connector as sql
import json, os
from tabulate import tabulate
from operator import itemgetter
import numpy as np
from dotenv import load_dotenv
load_dotenv()

conn = sql.connect(
    host = os.getenv("MYSQL_HOST"),
    user = os.getenv("MYSQL_USER"),
    password = os.getenv("MYSQL_PASSWORD"),
    database = 'DataAnalysis',
    port = '3306',
    use_pure = True)

cursor = conn.cursor(buffered=True , dictionary=True)

### Running Query

In [63]:
city = 'Sarnia'
query = ''' SELECT ZillowListings.Id, ZillowListings.Address, ZillowListings.CityName,
ZillowListings.ListingType, AVG(ZillowListingsAssociations.Price) AS Price
FROM ZillowListingsAssociations INNER JOIN ZillowListings ON 
ZillowListingsAssociations.Id = ZillowListings.Id
WHERE CityName = %(city)s AND Price > 1000
GROUP BY ZillowListings.Id'''
query_values = {'city': city}
cursor.execute(query, query_values)
result_set = cursor.fetchall()

### Visualizing Results

In [64]:
header = ['Id', 'Address', 'City', 'Type', 'Price', 'Status']
#Sort the Values by Price
dict_getter = itemgetter("Price")
result_set.sort(key=dict_getter, reverse=True)
#Creating the table form to visualize the data
values = []
price_list = []
for result in result_set:
    values.append(list(result.values()))
    price_list.append(result.get('Price'))
print(tabulate(values, headers=header))

        Id  Address                                          City    Type                      Price
----------  -----------------------------------------------  ------  -------------  ----------------
2056139970  1852 Lakeshore Rd, Sarnia, ON N7X 1C5            Sarnia  SINGLE_FAMILY       1.5999e+06
2055439549  613 Sweetbay Cres, Sarnia, ON N7S 0A8            Sarnia  SINGLE_FAMILY       1.2999e+06
2054926965  1757 Murphy Rd, Sarnia, ON N7S 2Z7               Sarnia  SINGLE_FAMILY       1.13562e+06
2054602574  601 Sweetbay Cres, Sarnia, ON N7S 0A8            Sarnia  SINGLE_FAMILY       1.13088e+06
2056301614  1607 Lakeshore Rd, Sarnia, ON N7X 1B5            Sarnia  SINGLE_FAMILY       1.1e+06
2059090741  268 Stathis Blvd, Sarnia, ON N7S 5T5             Sarnia  SINGLE_FAMILY       1e+06
2059092332  272 Stathis Blvd, Sarnia, ON N7S 5T5             Sarnia  SINGLE_FAMILY       1e+06
2054900882  500 Cherry Blossom Ct, Sarnia, ON N7S 0B2        Sarnia  SINGLE_FAMILY  970208
2054541591  394 Sa

### Finding Mean

In [65]:
mean_price = round(np.mean(price_list))
print(f'Mean Price of {len(price_list)} Zillow Listings: {mean_price}')

Mean Price of 74 Zillow Listings: 602462


### Finding Standard Deviation

In [66]:
standard_deviation = round(np.std(price_list))
print(f'Standard Deviation of {len(price_list)} Zillow Listings: {standard_deviation}')

Standard Deviation of 74 Zillow Listings: 279753


### The Emprical Rule

> According to the Emprical Rule: 
> + About 2/3 or 68% of the data fall within one standard deviation of the mean.
> + About 95% of the data fall within two standard deviation of the mean.
> + About 99.7% of the data fall within three standard deviation of the mean.

### Finding variance using single standard deviation of the provided data 

In [67]:
sd_minus_one = (mean_price - standard_deviation)
sd_plus_one = (mean_price + standard_deviation)
print(f'Low Two SD Range: {sd_minus_one}')
print(f'High Two SD Range: {sd_plus_one}')

Low Two SD Range: 322709
High Two SD Range: 882215


### Finding data that lies within single standard deviation

In [68]:
query = ''' SELECT ZillowListings.Id, ZillowListings.Address, ZillowListings.CityName,
ZillowListings.ListingType, AVG(ZillowListingsAssociations.Price) AS Price
FROM ZillowListingsAssociations INNER JOIN ZillowListings ON 
ZillowListingsAssociations.Id = ZillowListings.Id
WHERE CityName = %s AND Price > %s AND Price < %s
GROUP BY ZillowListings.Id'''
query_values = (city, sd_minus_one, sd_plus_one)
cursor.execute(query, query_values)
sd_one_result_set = cursor.fetchall()

header = ['Id', 'Address', 'City', 'Type', 'Price', 'Status']
#Sort the Values by Price
dict_getter = itemgetter("Price")
sd_one_result_set.sort(key=dict_getter)
#Creating the table form to visualize the data
sd_one_values = []
sd_one_price_list = []
for result in sd_one_result_set:
    sd_one_values.append(list(result.values()))
    sd_one_price_list.append(result.get('Price'))
print(tabulate(sd_one_values, headers=header))

        Id  Address                                          City    Type             Price
----------  -----------------------------------------------  ------  -------------  -------
2054503270  301 Edgewood St #8, Sarnia, ON N7S 4X7           Sarnia  CONDO           339000
2055032922  144 Mackenzie St S, Sarnia, ON N7T 3J3           Sarnia  SINGLE_FAMILY   349900
2054559584  306 Siddal St, Sarnia, ON N7T 3T4                Sarnia  SINGLE_FAMILY   358000
2055499431  185 Bedford Cres, Sarnia, ON N7S 4B4             Sarnia  TOWNHOUSE       369900
2054920448  273 Proctor St, Sarnia, ON N7T 3G9               Sarnia  SINGLE_FAMILY   373233
2056446111  2706 John St, Sarnia, ON N0N 1C0                 Sarnia  SINGLE_FAMILY   379000
2054920443  5700 Blackwell Side Rd #163, Sarnia, ON N7W 1B7  Sarnia  SINGLE_FAMILY   385000
2066849273  276 Cameron St, Sarnia, ON N7T 4B3               Sarnia  SINGLE_FAMILY   399000
2055208982  711 Murphy Rd, Sarnia, ON N7S 2X7                Sarnia  SINGLE_FAMI

In [69]:
sd_one_variance = len(sd_one_price_list)/len(price_list)
print(f'Amount of data found in single standard deviation of the mean: {sd_one_variance*100}%')

Amount of data found in single standard deviation of the mean: 72.97297297297297%


### Finding data that lies within two standard deviations

In [70]:
sd_minus_two = (mean_price - (2 * standard_deviation))
sd_plus_two = (mean_price + (2 * standard_deviation))
print(f'Low Two SD Range: {sd_minus_two}')
print(f'High Two SD Range: {sd_plus_two}')

Low Two SD Range: 42956
High Two SD Range: 1161968


### Finding data that lies within two standard deviations

In [71]:
query = ''' SELECT ZillowListings.Id, ZillowListings.Address, ZillowListings.CityName,
ZillowListings.ListingType, AVG(ZillowListingsAssociations.Price) AS Price 
FROM ZillowListingsAssociations INNER JOIN ZillowListings ON 
ZillowListingsAssociations.Id = ZillowListings.Id
WHERE CityName = %s AND Price > %s AND Price < %s
GROUP BY ZillowListings.Id'''
query_values = (city, sd_minus_two, sd_plus_two)
cursor.execute(query, query_values)
sd_two_result_set = cursor.fetchall()

header = ['Id', 'Address', 'City', 'Type', 'Price', 'Status']
#Sort the Values by Price
dict_getter = itemgetter("Price")
sd_two_result_set.sort(key=dict_getter, reverse=True)
#Creating the table form to visualize the data
sd_two_values = []
sd_two_price_list = []
for result in sd_two_result_set:
    sd_two_values.append(list(result.values()))
    sd_two_price_list.append(result.get('Price'))
print(tabulate(sd_two_values, headers=header))

        Id  Address                                          City    Type                      Price
----------  -----------------------------------------------  ------  -------------  ----------------
2054926965  1757 Murphy Rd, Sarnia, ON N7S 2Z7               Sarnia  SINGLE_FAMILY       1.13562e+06
2056301614  1607 Lakeshore Rd, Sarnia, ON N7X 1B5            Sarnia  SINGLE_FAMILY       1.1e+06
2059090741  268 Stathis Blvd, Sarnia, ON N7S 5T5             Sarnia  SINGLE_FAMILY       1e+06
2059092332  272 Stathis Blvd, Sarnia, ON N7S 5T5             Sarnia  SINGLE_FAMILY       1e+06
2054602574  601 Sweetbay Cres, Sarnia, ON N7S 0A8            Sarnia  SINGLE_FAMILY  999900
2054900882  500 Cherry Blossom Ct, Sarnia, ON N7S 0B2        Sarnia  SINGLE_FAMILY  970208
2054541591  394 Sawgrass Pl, Sarnia, ON N7W 0B2              Sarnia  SINGLE_FAMILY  949900
2059090739  316 Stathis Blvd, Sarnia, ON N7S 5T5             Sarnia  SINGLE_FAMILY  849900
2059090740  276 Stathis Blvd, Sarnia, ON N7S 5

In [72]:
sd_two_variance = len(sd_two_price_list)/len(price_list)
print(f'Amount of data found in single standard deviation of the mean: {sd_two_variance*100}%')

Amount of data found in single standard deviation of the mean: 95.94594594594594%


### Standardize The Data

> To compute areas unedr the normal curve, we first standardize the data by subtracting off x (mean) and then dividing by s (standard deviation):
> z = (data - x)/s

> Example: Find the z score of the data, for the Zillow listing with price CAD 200000

In [73]:
z = (300000 - mean_price)/standard_deviation
print(f'z score: {z}')

z score: -1.0811751795333742


### Normal Approximation

> Finding areas under the normal curve is called normal approximation.
> 
> Example: Finding the listings between the price range of CAD 300000 and CAD 700000
> + First we find the z score for the price CAD 300000
> + Second we find the z score for the price CAD 700000
> + Find the area between the two z scores

In [74]:
z1 = (300000 - mean_price)/standard_deviation
print(f'z score for CAD 300000: {z1}')
z2 = (700000 - mean_price)/standard_deviation
print(f'z score for CAD 700000: {z2}')

query = ''' SELECT ZillowListings.Id, ZillowListings.Address, ZillowListings.CityName,
ZillowListings.ListingType, AVG(ZillowListingsAssociations.Price) AS Price 
FROM ZillowListingsAssociations INNER JOIN ZillowListings ON 
ZillowListingsAssociations.Id = ZillowListings.Id
WHERE CityName = %s AND Price > %s AND Price < %s
GROUP BY ZillowListings.Id'''
query_values = (city, 300000, 700000)
cursor.execute(query, query_values)
z_area_result_set = cursor.fetchall()

#Sort the Values by Price
dict_getter = itemgetter("Price")
z_area_result_set.sort(key=dict_getter, reverse=True)
#Creating the table form to visualize the data
z_area_values = []
z_area_price_list = []
for result in z_area_result_set:
    z_area_values.append(list(result.values()))
    z_area_price_list.append(result.get('Price'))
#print(tabulate(z_area_values, headers=header))
z_score_area = (len(z_area_price_list)/len(price_list))*100
print(f'z score area: {z_score_area}%')

z score for CAD 300000: -1.0811751795333742
z score for CAD 700000: 0.34865756578124274
z score area: 58.108108108108105%
